In [15]:
# Install necessary libraries
!pip install numpy pandas scikit-learn tensorflow gym

# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import gym
from gym import spaces
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
from collections import deque

# Download the KDD Cup 1999 dataset

# Load the KDD Cup 99 dataset
df = pd.read_csv('Train.txt', header=None)

# Column names as per the KDD Cup 99 documentation
columns = ["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment",
           "urgent","hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted",
           "num_root","num_file_creations","num_shells","num_access_files","num_outbound_cmds",
           "is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
           "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate",
           "dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
           "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
           "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

df.columns = columns

# Encode categorical features
df['protocol_type'] = df['protocol_type'].astype('category').cat.codes
df['service'] = df['service'].astype('category').cat.codes
df['flag'] = df['flag'].astype('category').cat.codes
df['label'] = df['label'].apply(lambda x: 0 if x == 'normal.' else 1)  # Binary classification: normal vs attack

# Split the data
X = df.drop('label', axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the RL environment
class NIDSEnv(gym.Env):
    def __init__(self, X, y):
        super(NIDSEnv, self).__init__()
        self.X = X
        self.y = y
        self.action_space = spaces.Discrete(2)  # Two actions: 0 (normal), 1 (attack)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(X.shape[1],), dtype=np.float32)
        self.current_step = 0

    def reset(self):
        self.current_step = 0
        return self.X[self.current_step]

    def step(self, action):
        reward = 1 if action == self.y[self.current_step] else -1
        self.current_step += 1
        done = self.current_step >= len(self.X)
        info = {}
        if not done:
            next_state = self.X[self.current_step]
        else:
            next_state = np.zeros(self.X.shape[1])
        return next_state, reward, done, info

    def render(self, mode='human', close=False):
        pass

env = NIDSEnv(X_train, y_train)

# Define the DQN agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

agent = DQNAgent(state_size=X_train.shape[1], action_size=2)

# Train the DQN agent
episodes = 1000
batch_size = 32

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, agent.state_size])
    for time in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, agent.state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(f"episode: {e}/{episodes}, score: {time}, e: {agent.epsilon:.2}")
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    if e % 10 == 0:
        agent.save(f"dqn-nids-{e}.h5")

# Evaluate the model
def evaluate(env, agent, X_test, y_test):
    correct = 0
    for i in range(len(X_test)):
        state = np.reshape(X_test[i], [1, X_test.shape[1]])
        action = agent.act(state)
        if action == y_test[i]:
            correct += 1
    accuracy = correct / len(X_test)
    print(f"Accuracy: {accuracy:.4f}")

evaluate(env, agent, X_test, y_test)


ValueError: Length mismatch: Expected axis has 43 elements, new values have 42 elements